# Introduction

In [1]:
from ipywebcam import WebCamWidget, WebCamRecorder, Record, RecordPlayer, FileListFactory

In [2]:
factory = FileListFactory("test", "$i4-$Y-$m-$d-$H-${M}-${S}-$f-$uh6.mp4", condition=FileListFactory.create_frame_based_condition(100))

In [3]:
w = WebCamWidget(
    iceServers=[{"urls": "turn:190.92.221.222", "username": "admin", "credential": "123456"}],
    constraints={
        "video": {
            "frameRate": { "max": 10 },
            "width": 256,
            "height": 192,
        },
    },
)
w

WebCamWidget(constraints={'video': {'frameRate': {'max': 10}, 'width': 256, 'height': 192}}, iceServers=[{'url…

In [4]:
recorder = WebCamRecorder(w, factory)
recorder.start()

In [5]:
import cv2
from av import VideoFrame
async def transform(frame, ctx):
    img = frame.to_ndarray(format="bgr24")
    # prepare color
    img_color = cv2.pyrDown(cv2.pyrDown(img))
    for _ in range(6):
        img_color = cv2.bilateralFilter(img_color, 9, 9, 7)
    img_color = cv2.pyrUp(cv2.pyrUp(img_color))

    # prepare edges
    img_edges = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    img_edges = cv2.adaptiveThreshold(
        cv2.medianBlur(img_edges, 7),
        255,
        cv2.ADAPTIVE_THRESH_MEAN_C,
        cv2.THRESH_BINARY,
        9,
        2,
    )
    img_edges = cv2.cvtColor(img_edges, cv2.COLOR_GRAY2RGB)

    # combine color and edges
    img = cv2.bitwise_and(img_color, img_edges)

    # rebuild a VideoFrame, preserving timing information
    new_frame = VideoFrame.from_ndarray(img, format="bgr24")
    # new_frame.pts = frame.pts
    # new_frame.time_base = frame.time_base
    return new_frame

transformer = w.add_video_transformer(transform)

In [6]:
w.remove_video_transformer(transformer)

In [7]:
recorder.stop()

In [8]:
w.close_peers()

In [3]:
import logging
import cv2
from av import VideoFrame
logger = logging.getLogger('ipywebcam')
recorder = WebCamRecorder(None, factory)
recorder.factory.load()
print(recorder.factory.record_count())
player = RecordPlayer(recorder=recorder)
player.loop = True
def fix_time(frame, record, context):
    if "inited" not in context:
        context["inited"] = True
        if frame.pts > 0:
            offset = context["offset"] = frame.pts
        else:
            offset = 0
    else:
        offset = context.get("offset")
        offset = 0 if offset is None else offset
    if offset > 0:
        pass
        frame.pts -= offset
    return frame
#player.add_video_transformer(fix_time)

def transform(frame, r, ctx):
    iter = ctx.get('iter')
    if iter:
        iter = ctx["iter"] = iter + 1
    else:
        iter = ctx["iter"] = 0
    if iter % 10:
        print(iter)
    img = frame.to_ndarray(format="bgr24")
    # prepare color
    img_color = cv2.pyrDown(cv2.pyrDown(img))
    for _ in range(6):
        img_color = cv2.bilateralFilter(img_color, 9, 9, 7)
    img_color = cv2.pyrUp(cv2.pyrUp(img_color))

    # prepare edges
    img_edges = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    img_edges = cv2.adaptiveThreshold(
        cv2.medianBlur(img_edges, 7),
        255,
        cv2.ADAPTIVE_THRESH_MEAN_C,
        cv2.THRESH_BINARY,
        9,
        2,
    )
    img_edges = cv2.cvtColor(img_edges, cv2.COLOR_GRAY2RGB)

    # combine color and edges
    img = cv2.bitwise_and(img_color, img_edges)

    # rebuild a VideoFrame, preserving timing information
    new_frame = VideoFrame.from_ndarray(img, format="bgr24")
    # new_frame.pts = frame.pts
    # new_frame.time_base = frame.time_base
    return new_frame

transformer = player.add_video_transformer(transform, 'test')

load 59 records
59


In [6]:
recorder.factory.save()

In [8]:
player

RecordPlayer(loop=True, selected_range=[0.0, 0.0])

In [13]:
print(f"index: {player.selected_index}, channel: {player.selected_channel}, range: {player.selected_range}")

index: 0, channel: test, range: [4.0, 7.0]


In [8]:
a = { "a":1, "b":2}
print(a)
del a["b"]
print(a)

{'a': 1, 'b': 2}
{'a': 1}


In [11]:
from os import path
print(path.normpath('\\a\\b\\c'))
print(path.normpath('/a/b/c'))
p = path.relpath(path.normpath('a/b/c'), path.normpath('a/d'))
print(p)

\a\b\c
\a\b\c
..\b\c
